In [1]:
import sys
import os
import numpy as np
import pandas as pd

In [2]:
NORMALIZED_COLLECTION = "normalized_experts"

In [3]:
os.getcwd()

'/home/gil/dev/NEBULA2/fusion/playground'

In [4]:
os.chdir(os.getcwd()+'/../..')

In [5]:
from nebula_api.nebula_enrichment_api import *
from experts.common.RemoteAPIUtility import RemoteAPIUtility

In [6]:
nre = NRE_API()
api = RemoteAPIUtility()

In [7]:
mid = "Movies/92363218"

In [8]:
b = nre.get_clip_data(mid)
c = nre.get_vcomet_data(mid)

In [9]:
b[0]

[[['Slow motion shot of chauffeur helping businessman.',
   'The president is riding in a car.',
   'The prime minister is in a car.'],
  ['The organization leader arrives in an automobile.',
   'A criminal is sitting in a car.',
   'The president sits in his car.'],
  ['The businessmen are in a car.',
   'Men in black car.',
   'The businessmen are in a limousine.']],
 [[['bush', 'standing behind', 'plane'],
   ['driver', 'wearing a', 'tie'],
   ['head', 'sticking out of', 'car']],
  [['man', 'in a', 'car'],
   ['cop', 'in a', 'car'],
   ['driver', 'wearing a', 'tie']],
  [['driver', 'next to', 'passenger'],
   ['rear window', 'of', 'car'],
   ['driver', 'a', 'man']]]]

In [10]:
c[0]

{'_key': '131928667',
 '_id': 'nebula_vcomet_lighthouse/131928667',
 '_rev': '_dZisyiW---',
 'scene_element': 3,
 'start': 102,
 'stop': 231,
 'events': [[0.47332802414894104,
   ['is at the guardrails trying to get away',
    'is at the guardrails trying to get away']]],
 'places': [[0.4062037467956543, 'outside of two police cars at night'],
  [0.38419052958488464, 'outside cop cars'],
  [0.37296247482299805, 'outside a building with police cars'],
  [0.3687617778778076,
   'outside on a street at night that is filled with police cars'],
  [0.36111465096473694, 'outside next to a neon sign'],
  [0.35885727405548096, 'outside of a cop car']],
 'actions': [[0.5088728666305542,
   ['stop cars for id', 'They want to stop cars for id']],
  [0.48008376359939575,
   ['call the police on the speeding cars',
    'They want to call the police on the speeding cars']],
  [0.4759265184402466,
   ['hold the door open for the driver',
    'They want to hold the door open for the driver']],
  [0.472

In [11]:
def range_get_val_for_frame(frame,val,data):
    return [x[val] for x in data if x['start']<=frame and x['stop']>frame]
def tracker_get_box_for_frame(frame,data):
    return [x['bboxes'][str(frame)] for x in data if x['start']<=frame and x['stop']>frame]
def step_get_val_for_frame(frame, val,data):
    return [x[val] for x in data if int(x['frame_id'])==frame]

In [12]:
def normalize_movie(mid):
    action_data = nre.get_all_expert_data("Actions", mid)
    object_data = nre.get_all_expert_data("Object", mid)
    clip_data = nre.get_clip_data(mid)
    vcomet_data = nre.get_vcomet_data(mid)
    movie_info = api.get_movie_info(mid)
    frame_array = []
    for i in range(movie_info['last frame']):
        scene_element_val = range_get_val_for_frame(i,'scene_element', object_data)        
        frame_data = {'arango_id': mid, 'frame_id': i, 
                      'step_actor_id': step_get_val_for_frame(i, 'actor_id', action_data),
                      'step_description': step_get_val_for_frame(i, 'description', action_data),
                      'step_box': step_get_val_for_frame(i,'box', action_data),
                      'tracker_description': range_get_val_for_frame(i, 'description', object_data),
                      'scene_element': scene_element_val[0] if scene_element_val else None,
                      'tracker_box': tracker_get_box_for_frame(i, object_data) }                          
        scene = frame_data['scene_element']
        # Verify with dima what is order of scene in clip vs vcomet
        frame_data['clip_text'] = None
        frame_data['clip_triplets'] = None
        frame_data['vcomet_places'] = None
        frame_data['vcomet_events'] = None
        frame_data['vcomet_actions'] = None
        if scene is not None:
            frame_data['clip_text'] = clip_data[scene][0]
            frame_data['clip_triplets'] = clip_data[scene][1]
            frame_data['vcomet_places'] = range_get_val_for_frame(i,'places',vcomet_data)
            frame_data['vcomet_events'] = range_get_val_for_frame(i,'events',vcomet_data)
            frame_data['vcomet_actions'] = range_get_val_for_frame(i,'actions',vcomet_data)                    
        frame_array.append(frame_data)
    return frame_array                                         

In [21]:
def create_normalized_frame(frame):     
    query = 'UPSERT { arango_id: @arango_id, \
                                frame_id: @frame_id}\
            INSERT  \
                { arango_id: @arango_id, frame_id: @frame_id, step_actor_id: @step_actor_id, step_description: @step_description,  \
                  step_box: @step_box, tracker_description: @tracker_description, tracker_box: @tracker_box, scene_element: @scene_element, \
                  clip_text: @clip_text, clip_triplets: @clip_triplets, vcomet_places: @vcomet_places, vcomet_events: @vcomet_events, vcomet_actions: @vcomet_actions, \
                  updates: 1}\
            UPDATE \
                { updates: OLD.updates + 1, step_actor_id: @step_actor_id, step_description: @step_description,  \
                  step_box: @step_box, tracker_description: @tracker_description, tracker_box: @tracker_box, scene_element: @scene_element, \
                  clip_text: @clip_text, clip_triplets: @clip_triplets, vcomet_places: @vcomet_places, vcomet_events: @vcomet_events, vcomet_actions: @vcomet_actions} IN normalized_frames \
                    RETURN { doc: NEW, type: OLD ? \'update\' : \'insert\' }'       
    api.db.aql.execute(query, bind_vars=frame)

In [40]:
def get_normalized_frame(mid, frame):
    query = "FOR doc IN normalized_frames FILTER doc.arango_id == '{}' AND doc.frame_id == {} RETURN doc".format(mid,frame)
    print(query)
    cur = list(api.db.aql.execute(query))
    if cur:
        return cur[0]    # We only have one normalized frame (?)

In [46]:
def get_normalized_range(mid, start, stop):
    query = "FOR doc IN normalized_frames FILTER doc.arango_id == '{}' AND doc.frame_id >= {} AND doc.frame_id < {} RETURN doc".format(mid,start,stop)
    return list(api.db.aql.execute(query))    

In [51]:
def get_normalized_scene(mid, scene):
    query = "FOR doc IN normalized_frames FILTER doc.arango_id == '{}' AND doc.scene_element == {} RETURN doc".format(mid,scene)
    return list(api.db.aql.execute(query))        

In [42]:
a = normalize_movie(mid)

In [52]:
bla = get_normalized_scene(mid,1)

In [54]:
bla[-1]['frame_id']

44

In [29]:
a[84]['vcomet_places']

[[[0.37902727723121643, "in a limo driver's car"],
  [0.3553829491138458, 'in the police car'],
  [0.34957408905029297, "in 1's car"],
  [0.3299831748008728, 'in a police car']]]

In [84]:
action_data[5]

{'_key': '112457692',
 '_id': 'Nodes/112457692',
 '_rev': '_dBXv_ti---',
 'movie_id': 'cfe6da21c345473aa918ecb086a5d34c',
 'arango_id': 'Movies/92363218',
 'actor_id': None,
 'description': 'sit',
 'frame_id': '3',
 'class': 'Actions',
 'confidence': 0.8480625152587891,
 'updates': 1,
 'box': [0.4715381860733032,
  0.318683922290802,
  0.6713241338729858,
  0.6051863431930542]}

In [22]:
movie_info

{'arango_id': 'Movies/92363218',
 'description': 'actionclipautoautotrain00680',
 'fps': 25,
 'width': 592,
 'height': 320,
 'last frame': 300,
 'movie_id': 'cfe6da21c345473aa918ecb086a5d34c'}

In [87]:
object_data[0]

{'_key': '111197012',
 '_id': 'Nodes/111197012',
 '_rev': '_dBaFoMS---',
 'movie_id': 'cfe6da21c345473aa918ecb086a5d34c',
 'arango_id': 'Movies/92363218',
 'class': 'Object',
 'description': 'person0',
 'scores': {'0': 0.9361950159072876,
  '1': 0.9361950159072876,
  '2': 0.9361950159072876,
  '3': 0.9361950159072876,
  '4': 0.9361950159072876,
  '5': 0.9361950159072876,
  '6': 0.9361950159072876,
  '7': 0.9361950159072876,
  '8': 0.9361950159072876,
  '9': 0.9361950159072876,
  '10': 0.9986812472343445,
  '11': 0.9986812472343445,
  '12': 0.9986812472343445,
  '13': 0.9986812472343445,
  '14': 0.9986812472343445,
  '15': 0.9986812472343445,
  '16': 0.9986812472343445,
  '17': 0.9986812472343445,
  '18': 0.9986812472343445,
  '19': 0.9986812472343445,
  '20': 0.9977164268493652,
  '21': 0.9977164268493652,
  '22': 0.9977164268493652,
  '23': 0.9977164268493652,
  '24': 0.9977164268493652,
  '25': 0.9977164268493652,
  '26': 0.9977164268493652,
  '27': 0.9977164268493652,
  '28': 0.9977

In [58]:
len(tracker_get_val_for_frame(44,'description')), len(tracker_get_box_for_frame(44))

(14, 14)

In [13]:
api.get_expert_status("Actions",'movies')

False

In [13]:
api.db.create_collection("giltest")

<StandardCollection giltest>

In [16]:
nre.force_start_expert("Actions")

Updating global version


In [23]:
new_movies = api.get_new_movies()
for x in new_movies[1:]:
    api.change_status_movie('updated', x)

In [24]:
api.get_new_movies()

['Movies/114206264']